In [14]:
import sys
!{sys.executable} -m pip install altair
!{sys.executable} -m pip install altair_data_server
!{sys.executable} -m pip install prince
!{sys.executable} -m pip install xgboost
!{sys.executable} -m pip install tensorflow
!{sys.executable} -m pip install keras
!{sys.executable} -m pip install scikeras

import pandas as pd
import numpy as np
import altair as alt
alt.data_transformers.enable('data_server')

DataTransformerRegistry.enable('data_server')

In [15]:
default_df = pd.read_excel('assets/default of credit card clients.xls', header=1, index_col=0)
default_df.rename(columns={'default payment next month': 'default', 'PAY_0': 'PAY_1'}, inplace=True)
default_df.head()

,LIMIT_BAL,SEX,EDUCATION,MARRIAGE,AGE,PAY_1,PAY_2,PAY_3,PAY_4,PAY_5,...,BILL_AMT4,BILL_AMT5,BILL_AMT6,PAY_AMT1,PAY_AMT2,PAY_AMT3,PAY_AMT4,PAY_AMT5,PAY_AMT6,default
ID,,,,,,,,,,,,,,,,,,,,,
1,20000,2,2,1,24,2,2,-1,-1,-2,...,0,0,0,0,689,0,0,0,0,1
2,120000,2,2,2,26,-1,2,0,0,0,...,3272,3455,3261,0,1000,1000,1000,0,2000,1
3,90000,2,2,2,34,0,0,0,0,0,...,14331,14948,15549,1518,1500,1000,1000,1000,5000,0
4,50000,2,2,1,37,0,0,0,0,0,...,28314,28959,29547,2000,2019,1200,1100,1069,1000,0
5,50000,1,2,1,57,-1,0,-1,0,0,...,20940,19146,19131,2000,36681,10000,9000,689,679,0


In [16]:
tot_default = default_df.default.sum()
tot_non_default = default_df.shape[0] - default_df.default.sum()
pd.DataFrame({'Total default': [tot_default], 'Total non-default': [tot_non_default],
              'Non-default/default ratio': [round(tot_non_default / tot_default, 2)]})

,Total default,Total non-default,Non-default/default ratio
0,6636,23364,3.52


Unexpected discrepancies in the dataset description:

--   **'0', '5' and '6'** EDUCATION levels are not mentioned in the dataset description document
--   **'0'** MARRIAGE level is not mentioned in the dataset description document
--   **'-2' and '0'** PAY_X levels are not mentioned in the dataset description document

In [17]:
education_levels = default_df.EDUCATION.unique()
education_levels.sort()
marriage_levels = default_df.MARRIAGE.unique()
marriage_levels.sort()
payment_status_levels = default_df.PAY_1.unique()
payment_status_levels.sort()
print('Levels of EDUCATION: ', education_levels)
print('Levels of MARRIAGE: ', marriage_levels)
print('Levels of PAY_X: ', payment_status_levels)

Levels of EDUCATION:  [0 1 2 3 4 5 6]
Levels of MARRIAGE:  [0 1 2 3]
Levels of PAY_X:  [-2 -1  0  1  2  3  4  5  6  7  8]


Let's see what distribution of ages of credit card clients is

In [18]:
alt.Chart(default_df, title='Distribution of ages of credit card clients').mark_bar().encode(
    x=alt.X("AGE:Q", bin=alt.BinParams(maxbins = 30)),
    y='count()'
)

alt.Chart(...)

In [19]:
alt.Chart(default_df, width=300, title='Ages of default vs. non-default credit card clients').mark_boxplot(size=50, extent=1.5).encode(
    x=alt.X('default:O'),
    y=alt.Y('AGE:Q', scale=alt.Scale(zero=False))
)

alt.Chart(...)

## Data transformation

The following experimental data transformation steps were made:

1. One-hot encode all categorical variables using pandas' get_dummies() function - a binary variable is created for each level of categorical variables.
2. History of past payments variables are encoded using the following rule:
*   (value <= 0) -> PAY_x_DUE = 'YES' (where x is in [1,2,3,4,5,6])
*   (value > 0) -> PAY_x_DUE = 'NO' (where x is in [1,2,3,4,5,6])
     New variables are then one-hot encoded.
3. AGE variable is converted into AGE_GROUP (number of full decades in age, eg. 23yo => age group 2, 58yo => age group 5, etc.) and then one-hot encoded as well.

In [20]:
# data transformation
def_cat_df = default_df.copy()
def_cat_df['AGE_GROUP'] = (def_cat_df.AGE / 10).apply(np.floor).astype(int)
def_cat_df['PAY_1_DUE'] = np.where(def_cat_df['PAY_1'] <= 0, 'YES', 'NO')
def_cat_df['PAY_2_DUE'] = np.where(def_cat_df['PAY_2'] <= 0, 'YES', 'NO')
def_cat_df['PAY_3_DUE'] = np.where(def_cat_df['PAY_3'] <= 0, 'YES', 'NO')
def_cat_df['PAY_4_DUE'] = np.where(def_cat_df['PAY_4'] <= 0, 'YES', 'NO')
def_cat_df['PAY_5_DUE'] = np.where(def_cat_df['PAY_5'] <= 0, 'YES', 'NO')
def_cat_df['PAY_6_DUE'] = np.where(def_cat_df['PAY_6'] <= 0, 'YES', 'NO')
def_cat_df = pd.get_dummies(def_cat_df, columns=['PAY_1_DUE', 'PAY_2_DUE', 'PAY_3_DUE', 'PAY_4_DUE', 'PAY_5_DUE', 'PAY_6_DUE'], drop_first=True)
def_cat_df.drop(columns=['PAY_1', 'PAY_2', 'PAY_3', 'PAY_4', 'PAY_5', 'PAY_6', 'AGE'], inplace=True)
def_cat_df['SEX'] = np.where(def_cat_df['SEX'] == 1, 'MALE', 'FEMALE')
def_cat_df = pd.get_dummies(def_cat_df, columns=['SEX'], drop_first=True)
def_cat_df = pd.get_dummies(def_cat_df, columns=['EDUCATION', 'MARRIAGE', 'AGE_GROUP'])
def_cat_df.head()

,LIMIT_BAL,BILL_AMT1,BILL_AMT2,BILL_AMT3,BILL_AMT4,BILL_AMT5,BILL_AMT6,PAY_AMT1,PAY_AMT2,PAY_AMT3,...,MARRIAGE_0,MARRIAGE_1,MARRIAGE_2,MARRIAGE_3,AGE_GROUP_2,AGE_GROUP_3,AGE_GROUP_4,AGE_GROUP_5,AGE_GROUP_6,AGE_GROUP_7
ID,,,,,,,,,,,,,,,,,,,,,
1,20000,3913,3102,689,0,0,0,0,689,0,...,0,1,0,0,1,0,0,0,0,0
2,120000,2682,1725,2682,3272,3455,3261,0,1000,1000,...,0,0,1,0,1,0,0,0,0,0
3,90000,29239,14027,13559,14331,14948,15549,1518,1500,1000,...,0,0,1,0,0,1,0,0,0,0
4,50000,46990,48233,49291,28314,28959,29547,2000,2019,1200,...,0,1,0,0,0,1,0,0,0,0
5,50000,8617,5670,35835,20940,19146,19131,2000,36681,10000,...,0,1,0,0,0,0,0,1,0,0
